In [ ]:
import pandas as pd
from shapely.geometry import Point, Polygon
import os
import tkinter as tk
from tkinter import filedialog

In [ ]:
def load_cell_polygons(cell_boundary_folder):
    cell_polygons = {}
    for cell_file in os.listdir(cell_boundary_folder):
        if cell_file.endswith(".txt"):
            fov_name_and_cell = cell_file.replace(".txt", "")
            fov_name,_,cell_num = fov_name_and_cell.repartition('_cell')
            if fov_name not in cell_polygons:
                cell_polygons[fov_name] = []
            cell_outline_coordinates = pd.read_csv (os.path.join(cell_boundary_folder, cell_file), sep="\t", header=None)

            coords_roi = [tuple(row) for _, row in cell_outline_coordinates.iterrows()]
            cell_polygon = Polygon(coords_roi)
            cell_polygons[fov_name].append((cell_polygon, cell_num))
    return cell_polygons            

In [ ]:
def assign_cellID_to_tracks(track_df, cell_polygons_dict):
    track_df['cellID'] = 'cell1'
    for fov_name, polygons_info in cell_polygons_dict.items():
        tracks_in_fov = track_df[track_df['fov_name'] == fov_name]
        for index, track in tracks_in_fov.iterrows():
            point = Point(track['x_mean'], track['y_mean'])
            for polygon, cell_id in polygons_info:
                if point.within(polygon):
                    track_df.at[index,'cellID'] = f"cell{cell_id}"
                    break
    
    return track_df

In [ ]:
if __name__=="__main__":
    root = tk.Tk()
    root.withdraw

In [ ]:
cell_boundary_folder = filedialog.askdirectory(title="Select folder containing cell boundary files")

In [ ]:
cell_polygons_dict = load_cell_polygons (cell_boundary_folder)

In [ ]:
csv_file_path = filedialog.askdirectory(
    title="Select the track CSV File"
    filetypes=(("CSV files", "*.csv")),
)

In [ ]:
track_df = pd.read_csv(csv_file_path)

In [ ]:
track_df['x_mean'] = track_df.groupby('fov_name')['x'].transform('mean')
track_df['y_mean'] = track_df.groupby('fov_name')['y'].transform('mean')

In [ ]:
track_df_with_cellID = assign_cellID_to_tracks(track_df, cell_polygons_dict)

In [ ]:
output_file_path = csv_file_path.replace(".csv", "_with_CellIDs.csv")
track_df_with_cellID.to_csv(output_file_path,index=False)
print(f"Updated track data with cell IDs has been saved to: {output_file_path}")